# AutoEncoders

## Recommendation System for non binary prediction.

In [26]:
import numpy as np
import pandas as pd
import pyodbc

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [27]:
movies = pd.read_csv('dataset/movies.dat',sep="::",header=None,engine='python',encoding='latin-1')
users = pd.read_csv('dataset/users.dat',sep="::",header=None,engine='python',encoding='latin-1')
ratings = pd.read_csv('dataset/ratings.dat',sep="::",header=None,engine='python',encoding='latin-1')

In [28]:
training_set = np.array(pd.read_csv('dataset/u1.base',delimiter='\t'),dtype='int')
test_set = np.array(pd.read_csv('dataset/u1.test',delimiter='\t'),dtype='int')

In [29]:
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))

In [30]:
def convert(data):
    new_data=[]
    for id_users in np.arange(1,nb_users+1):
        id_movies = data[:,1][data[:,0]==id_users]
        id_ratings = data[:,2][data[:,0]==id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [31]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [32]:
class SAE(nn.Module):
    def __init__(self,):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [33]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay = 0.5)

In [ ]:
nb_epoch = 200
for epoch in np.arange(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in np.arange(nb_users):
        input_vec = Variable(training_set[id_user]).unsqueeze(0)
        target_vec = input_vec.clone()
        if torch.sum(target_vec.data > 0) > 0:
            output_vec = sae(input_vec)
            target_vec.require_grad = False
            output_vec[target_vec == 0] = 0
            loss = criterion(output_vec, target_vec)
            mean_corrector = nb_movies/float(torch.sum(target_vec.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1
            optimizer.step()
    print('epoch: ' + str(epoch)+' loss: '+str(train_loss/s))

In [ ]:
test_loss = 0
s = 0.
for id_user in np.arange(nb_users):
    input_vec = Variable(training_set[id_user]).unsqueeze(0)
    target_vec = input_vec.clone()
    if torch.sum(target_vec.data > 0) > 0:
        output_vec = sae(input_vec)
        target_vec.require_grad = False
        output_vec[target_vec == 0] = 0
        loss = criterion(output_vec, target_vec)
        mean_corrector = nb_movies/float(torch.sum(target_vec.data > 0) + 1e-10)
        loss.backward()
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1
        optimizer.step()